In [86]:
import sklearn
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score
import random

In [3]:
target_df = pd.read_csv("../results/multiclass_data.csv", sep="\t")
target_df.head()

,UniProt_ID,membrane_bound,cytoplasm,Nucleus,Mitochondria,Vesicles,Golgi_apparatus,Centrosome,extra_cellular,Cytosol,...,Nucleoli,Nucleoplasm,cyto_skeleton,Intermediate_filaments,Nuclear_speckles,Focal_adhesion_sites,Cytokinetic_bridge,Lysosomes,Endoplasmic_reticulum,sequence
0,Q9Y5Q8,0,0,1,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,MAAEAADLGLGAAVPVELRRERRMVCVEYPGVVRDVAKMLPTLGGE...
1,Q9NZP6,0,0,1,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,MGNLLSKFRPGCRRRPLPGPGRGAPAPLSRDASPPGRAHSVPTPRP...
2,Q9Y2Q5,0,0,1,0,1,0,0,1,0,...,0,1,0,0,0,0,0,0,0,MLRPKALTQVLSQANTGGVQSTLLLNNEGSLLAYSGYGDTDARVTA...
3,O43155,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,MGLQTTKWPSHGAFFLKSWLIISLGLYSQVSKLLACPSVCRCDRNF...
4,O75954,1,1,1,0,0,1,0,0,1,...,0,1,0,0,0,0,0,0,0,MARGCLCCLKYMMFLFNLIFWLCGCGLLGVGIWLSVSQGNFATFSP...


In [7]:
feature_df = pd.read_csv("../results/multiclass_data_prot_vec.csv", sep="\t", header=None, index_col=0)
feature_df.head()

,1,2,3,4,5,6,7,8,9,10,...,91,92,93,94,95,96,97,98,99,100
0,,,,,,,,,,,,,,,,,,,,,
Q9Y5Q8,-34.153008,-6.904771,0.611060,-36.184279,-1.443093,-10.265217,18.400099,-2.779048,-7.155600,35.489984,...,11.464924,-14.204053,-12.138947,-0.347427,22.600651,3.524877,-4.967677,-29.510203,-10.069064,32.076692
Q9NZP6,-90.341616,-30.626979,-19.577185,-100.685267,-30.814087,-23.925996,14.476417,-38.476923,-7.125383,77.520812,...,28.007054,-41.634869,-46.003616,42.833975,53.641541,23.968170,-4.436353,-122.918051,-18.788588,82.180068
Q9Y2Q5,-8.575628,-1.246332,2.937982,-6.926931,-2.638474,-3.306731,1.885642,-1.516034,-1.178943,7.447700,...,3.554990,-5.870313,-1.262408,0.487179,3.753397,0.938354,-2.857358,-4.071456,-4.303386,9.460072
O43155,-44.960466,-10.580352,-19.184198,-54.501819,13.053520,-4.167851,15.741654,-17.513083,-22.085207,40.304068,...,3.814329,-12.778699,-18.156404,0.585945,24.665101,6.751130,-8.297106,-42.041376,-9.209458,50.469912
O75954,-12.625779,1.804811,-3.891998,-23.965740,6.909728,-2.713406,3.241422,-7.937678,-9.257087,8.203030,...,3.490328,-12.108776,-10.128157,-4.999490,1.495985,2.437419,-9.397000,-12.283494,-10.600909,31.695490


In [21]:
target = target_df.iloc[:,2:-1]
target_labels = target.columns
y = target.values
target_labels, y

(Index(['cytoplasm', 'Nucleus', 'Mitochondria', 'Vesicles', 'Golgi_apparatus',
        'Centrosome', 'extra_cellular', 'Cytosol', 'Endosomes', 'Peroxisomes',
        'Actin_filaments', 'Nucleoli_fibrillar_center', 'Nuclear_bodies',
        'Plasma_membrane', 'Nuclear_membrane', 'Cell_Junctions',
        'Microtubule_organizing_center', 'Midbody', 'Microtubules', 'Nucleoli',
        'Nucleoplasm', 'cyto_skeleton', 'Intermediate_filaments',
        'Nuclear_speckles', 'Focal_adhesion_sites', 'Cytokinetic_bridge',
        'Lysosomes', 'Endoplasmic_reticulum'],
       dtype='object'), array([[0, 1, 0, ..., 0, 0, 0],
        [0, 1, 0, ..., 0, 0, 0],
        [0, 1, 0, ..., 0, 0, 0],
        ..., 
        [0, 0, 0, ..., 0, 0, 0],
        [0, 1, 0, ..., 0, 0, 0],
        [1, 1, 0, ..., 0, 0, 0]]))

In [20]:
X = feature_df.values
X

array([[ -34.153008,   -6.904771,    0.61106 , ...,  -29.510203,
         -10.069064,   32.076692],
       [ -90.341616,  -30.626979,  -19.577185, ..., -122.918051,
         -18.788588,   82.180068],
       [  -8.575628,   -1.246332,    2.937982, ...,   -4.071456,
          -4.303386,    9.460072],
       ..., 
       [ -64.976319,   -8.52096 ,  -13.677441, ...,  -50.769369,
         -10.576738,   81.706415],
       [ -20.398099,   -9.368096,   -2.772432, ...,  -30.8018  ,
          -3.207864,    9.145268],
       [ -38.983716,   -1.609826,   -0.639653, ...,  -37.356834,
           0.568018,   35.025829]])

## stratified K-fold

hack: consider protein as a random location of the given locations


In [77]:
random.seed(42)

In [78]:
def strata_multilabel(y):
    """hack: chooses a random label for each entry of the actual labels
    which is used for stratification"""
    return np.argmax(y + np.random.normal(0, 0.01, y.shape), axis=1)

In [79]:
# split into independent test set and CV
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=strata_multilabel(y))

In [80]:
N_FOLDS = 5

In [92]:
skf = StratifiedKFold(n_splits=N_FOLDS, shuffle=False)
model = ExtraTreesClassifier(200, random_state=42, n_jobs=-1)

In [93]:
accuracy = np.empty(N_FOLDS)
precision = np.empty((target_labels.size, N_FOLDS))
recall = np.empty((target_labels.size, N_FOLDS))

In [ ]:
for i, (train_ind, test_ind) in enumerate(skf.split(X_train, strata_multilabel(y_train))):
    print("RUNNING FOLD {}".format(i))
    model.fit(X_train[train_ind], y_train[train_ind])
    predicted = model.predict(X_train[test_ind])
    truth = y_train[test_ind]
    acc = accuracy_score(truth, predicted)
    accuracy[i] = acc
    print("Overall Accuracy {}".format(acc))
    for j, label in enumerate(target_labels):
        true_class = truth[:,j]
        predicted_class = predicted[:,j]
        prec = precision_score(true_class, predicted_class)
        rec = recall_score(true_class, predicted_class)
        precision[j,i] = prec
        recall[j, i] = rec

RUNNING FOLD 0


Average Accuracy

In [ ]:
accuracy.mean()

In [ ]:
avg_recall = recall.mean(axis=1)
avg_precision =precision.mean(axis=1)
pd.DataFrame()\
    .assign(compartment=target_labels, precision=avg_precision, recall=avg_recall )